# Agent and Agent Runtime

In this and the following section, we focus on the 
[core concepts](../core-concepts/overview.md) of AGNext:
agents, agent runtime, messages, and communication.
You will not find any AI models or tools here, just the foundational
building blocks for building multi-agent applications.

An agent in AGNext is an entity defined by the base class {py:class}`agnext.core.BaseAgent`.
It has a unique identifier of the type {py:class}`agnext.core.AgentId`,
a metadata dictionary of the type {py:class}`agnext.core.AgentMetadata`,
and method for handling messages {py:meth}`agnext.core.BaseAgent.on_message`.

An agent runtime is the execution environment for agents in AGNext.
Similar to the runtime environment of a programming language,
an agent runtime provides the necessary infrastructure to facilitate communication
between agents, manage agent lifecycles, enforce security boundaries, and support monitoring and
debugging.
For local development, developers can use {py:class}`~agnext.application.SingleThreadedAgentRuntime`,
which can be embedded in a Python application.

```{note}
Agents are not directly instantiated and managed by application code.
Instead, they are created by the runtime when needed and managed by the runtime.
```

## Implementing an Agent

To implement an agent, developer must subclass the {py:class}`~agnext.core.BaseAgent` class,
declare the message types it can handle in the {py:attr}`~agnext.core.AgentMetadata.subscriptions` metadata,
and implement the {py:meth}`~agnext.core.BaseAgent.on_message` method.
This method is invoked when the agent receives a message. For example,
the following agent handles a simple message type and simply prints message it receives:

In [2]:
from dataclasses import dataclass
from agnext.core import BaseAgent, CancellationToken

@dataclass
class MyMessage:
    content: str

class MyAgent(BaseAgent):
    def __init__(self):
        super().__init__("MyAgent", subscriptions=[MyMessage])

    async def on_message(self, message: MyMessage, cancellation_token: CancellationToken) -> None:
        print(f"Received message: {message.content}")

For convenience, developers can subclass the {py:class}`~agnext.components.TypeRoutedAgent` class
which provides an easy-to use API to implement different message handlers for different message types.
See the section on message handlers below.

## Registering Agents

To make an agent available to the runtime, developers can use the
{py:meth}`~agnext.core.AgentRuntime.register` method.
The process of registration associates a name and a factory function
that creates an instance of the agent in a given namespace.
The factory function is used to allow automatic creation of agents when they are needed.

For example, to register an agent with the {py:class}`~agnext.application.SingleThreadedAgentRuntime`,
the following code can be used:

In [4]:
from agnext.application import SingleThreadedAgentRuntime

runtime = SingleThreadedAgentRuntime()
await runtime.register("my_agent", lambda: MyAgent())

Once an agent is registered, the agent's unique ID of the type {py:class}`~agnext.core.AgentId` 
can be retrieved by calling {py:meth}`~agnext.core.AgentRuntime.get`.
Using the agent ID, we can send messages to the agent:

In [ ]:
agent_id = await runtime.get("my_agent")
run_context = runtime.start() # Start processing messages in the background.
await runtime.send_message(MyMessage(content="Hello, World!"), agent_id)
await run_context.stop() # Stop processing messages in the background.

Received message: Hello, World!


There is a convenience method
{py:meth}`~agnext.core.AgentRuntime.register_and_get` that both registers an agent
and gets its ID.

```{note}
Because the runtime manages the lifecycle of agents, a reference to an agent,
whether it is {py:class}`~agnext.core.AgentId` or {py:class}`~agnext.core.AgentProxy`,
is only used to communicate with the agent or retrieve its metadata (e.g., description).
```

## Running the Single-Threaded Agent Runtime

The above code snippet uses `runtime.start()` to start a background task
to process and deliver messages to recepients' message handlers.
This is a feature of the
local embedded runtime {py:class}`~agnext.application.SingleThreadedAgentRuntime`.

To stop the background task immediately, use the `stop()` method:

In [6]:
run_context = runtime.start()
# ... Send messages, publish messages, etc.
await run_context.stop() # This will return immediately but will not cancel
# any in-progress message handling.

You can resume the background task by calling `start()` again.

For batch scenarios such as running benchmarks for evaluating agents,
you may want to wait for the background task to stop automatically when
there are no unprocessed messages and no agent is handling messages --
the batch may considered complete.
You can achieve this by using the `stop_when_idle()` method:

In [7]:
run_context = runtime.start()
# ... Send messages, publish messages, etc.
await run_context.stop_when_idle() # This will block until the runtime is idle.

You can also directly process messages one-by-one without a background task using:

In [8]:
await runtime.process_next()

Other runtime implementations will have their own ways of running the runtime.